# Ames Housing Prices - Step 2: Data Prep
In this phase, we will build a data prep pipeline to handle outliers and missing data.

In [ ]:
# Basic setup
%run ./config.ipynb

In [ ]:
# Connect to Cortex 5
cortex = Cortex.client()

In [ ]:
# Retrieve training dataset from Cortex
train_ds = cortex.dataset('kaggle/ames-housing-train')

## Pipelines
Each Cortex 5 Dataset can have multiple named pipelines which are chains of Python functions that transfor the dataset for specific purposes (e.g. data cleaning, feature prep, feature framing, etc.).  We begin here by creating a _clean_ pipeline that will handle remove unwanted columns, handle missing data, remove outliers.

In [ ]:
pipeline = train_ds.pipeline('clean', clear_cache=True)
pipeline.reset()

### Pipeline Steps
Each step in the pipeline is a Python function that accepts and instance of the pipeline object and a Pandas DataFrame for processing.

In [ ]:
def drop_unused(pipeline, df):
    df.drop(columns=['Id'], axis=1, inplace=True)

pipeline.add_step(drop_unused)

In [ ]:
def drop_outliers(pipeline, df):
    df.drop(df[(df['GrLivArea'].astype(int)>4000) & (df['SalePrice'].astype(int)<300000)].index, inplace=True)
    
pipeline.add_step(drop_outliers)

In [ ]:
def fill_zero_cols(pipeline, df):
    fill_zero_cols = ['BsmtHalfBath', 'BsmtFullBath', 'BsmtFinSF2', 'GarageCars']
    [df[i].fillna(0, inplace=True) for i in fill_zero_cols]

pipeline.add_step(fill_zero_cols)

### Using the Pipeline Memory
Each pipeline has a memory, called _context_ that is useful for recording state that needs to be used later.  In this case, we are recording the median values of several columns that will be used to fill in missing values for those columns.  We need these median values later when encounter test instances and need to fill in the same missing values.

In [ ]:
def fill_median_cols(pipeline, df):
    fill_median_cols = ['GarageArea','TotalBsmtSF', 'MasVnrArea', 'BsmtFinSF1', 'LotFrontage', 'BsmtUnfSF', 'GarageYrBlt']
    [pipeline.set_context('{}_median'.format(j), np.asscalar(df[j].astype(float).median())) for j in fill_median_cols]
    [df[j].fillna(df[j].astype(float).median(), inplace=True) for j in fill_median_cols]
    
pipeline.add_step(fill_median_cols)

In [ ]:
def fill_na_none(pipeline, df):
    df.fillna('none', inplace=True)

pipeline.add_step(fill_na_none)

### Running the Pipeline
The pipeline _run_ method will run each of our steps in order and return a transformed DataFrame instance.

In [ ]:
train_df = pipeline.run()

---
Lets validate that our pipeline solved our missing data and outliers problem.

In [ ]:
viz = train_ds.visuals(train_df, figsize=(24, 9))

In [ ]:
viz.show_missing()

In [ ]:
viz.show_corr_pairs('SalePrice', threshold=0.7)